In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import neurokit2 as nk
from datetime import datetime
import pandas as pd

In [3]:
data = pd.read_csv('C:/Users/eleon/Desktop/SDAP/backend/src/data/p1_wk1/1022102cFnorm.csv', usecols=['MR', 'ML', 'ECG'])
loc = pd.read_csv('C:/Users/eleon/Desktop/SDAP/backend/src/data/p1_wk1/1022102clocation_Bites.csv')

In [4]:
data_5_min = data[:600000]

In [5]:
data_5_min

,MR,ML,ECG
0,0.060487,-0.016969,0.234100
1,0.658350,-0.184600,0.234080
2,3.215500,-0.900960,0.234100
3,9.209700,-2.577700,0.234110
4,16.877000,-4.715300,0.234090
...,...,...,...
599995,0.000559,-0.013620,0.000705
599996,-0.000154,-0.002426,0.000699
599997,0.006617,0.005797,0.000696
599998,0.009843,-0.000941,0.000694


# Rectified signal

In [6]:
rectified_signals = np.abs(data_5_min)

# RMS

In [7]:
# Define a window size (in number of samples)
# 60-120 ms?
window_size = int(0.070*2000)

# Compute RMS with a rolling window
rms = rectified_signals.rolling(window=window_size, min_periods=1).apply(lambda x: np.sqrt(np.mean(x**2)), raw=True)

# Check the output size
print(f"Original signal length: {len(rectified_signals)}")
print(f"RMS signal length: {len(rms)}")

Original signal length: 600000
RMS signal length: 600000


In [8]:
rms

,MR,ML,ECG
0,0.060487,0.016969,0.234100
1,0.467484,0.131082,0.234090
2,1.895303,0.531066,0.234093
3,4.888637,1.368451,0.234098
4,8.722709,2.438224,0.234096
...,...,...,...
599995,0.028857,0.015691,0.000710
599996,0.028824,0.015638,0.000710
599997,0.028822,0.015505,0.000710
599998,0.028831,0.015255,0.000710


# FIND MVC (Maximum Volontary Contraction)

In [9]:
# Define Maximum Volontary Contraction
mvc_MR = 0
mvc_ML = 0
for index, row in loc.iterrows():
    begin_contraction, end_contraction = int(row[0]), int(row[1])


    bite_data = rms[begin_contraction:end_contraction]

    max_in_bite_MR = bite_data['MR'].max()
    max_in_bite_ML = bite_data['ML'].max()

    print(max_in_bite_MR)
    print(max_in_bite_ML)


    # TODO: check if average of ML and MR max row is okay
    # max_in_bite_avg = (max_in_bite['MR'][0] + max_in_bite['ML'][0]) / 2

    if float(max_in_bite_MR) > float(mvc_MR):
        mvc_MR = max_in_bite_MR

    if float(max_in_bite_ML) > float(mvc_ML):
        mvc_ML = max_in_bite_ML
    

0.9604318320957134
1.4121848163703676
1.0498430017794986
1.6142088837512722
0.6841394044142702
1.6648890864077919


C:\Users\eleon\AppData\Local\Temp\ipykernel_23956\2061933194.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  begin_contraction, end_contraction = int(row[0]), int(row[1])


In [10]:
mvc_MR, mvc_ML

(1.0498430017794986, 1.6648890864077919)

# DOWNSAMPLING

In [11]:
data_MR_5_min_resampled = nk.signal_resample(rms['MR'], sampling_rate=2000, desired_sampling_rate=256, method="interpolation")
data_ML_5_min_resampled = nk.signal_resample(rms['ML'], sampling_rate=2000, desired_sampling_rate=256, method="interpolation")
data_ECG_5_min_resampled = nk.signal_resample(rms['ECG'], sampling_rate=2000, desired_sampling_rate=256, method="interpolation")

In [21]:
len(data_MR_5_min_resampled)

76800

In [12]:
data_ECG_5_min_resampled

array([0.2341    , 0.23409428, 0.23409278, ..., 0.00070296, 0.00070851,
       0.00070941])

In [14]:
signals, info = nk.ecg_process(data_ECG_5_min_resampled, sampling_rate=256)

In [17]:
rate = signals["ECG_Rate"]

70.19944552273927

In [19]:
data_5_min_resampled = pd.DataFrame(data={'MR': data_MR_5_min_resampled, 'ML': data_ML_5_min_resampled, 'ECG': data_ECG_5_min_resampled})

In [20]:
data_5_min_resampled

,MR,ML,ECG
0,0.060487,0.016969,0.234100
1,10.806310,3.006534,0.234094
2,8.628713,2.395300,0.234093
3,8.865398,2.489589,0.234067
4,8.552050,2.407505,0.234041
...,...,...,...
76795,0.028353,0.017080,0.000708
76796,0.028061,0.016840,0.000705
76797,0.028103,0.016733,0.000703
76798,0.028696,0.016020,0.000709


In [23]:
data_5_min_resampled.to_csv('../../frontend/src/assets/1022102cFnorm_rms_5_min_256Hz.csv', )